<a href="https://colab.research.google.com/github/osherlock1/AI-Image-Detection-Model/blob/main/main_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Start with a Resnet Model, Try ViTs if we have time**

## **Imports**

In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

from google.colab import drive

## **Import Dataset**

In [25]:
# prompt: connect to google drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
data_dir = "/content/drive/MyDrive/URI_spr25/ELE392/AI_Detection_Model/Datasets/Processed_Dataset"  # Update if needed


In [26]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root=data_dir, transform=transform)

**Create DataLoaders**

In [27]:
#Split Datset

train_len = int(len(dataset) * 0.8)
val_len = int(len(dataset) * 0.1)
test_len = len(dataset) - train_len - val_len # test_len will automatically be an int

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_len, val_len, test_len])

In [28]:
#Create DataLoaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle=True);
test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

## **Train with Resnet Model**

In [29]:
#Define Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#Use Resnset18 for now
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 1)  # Binary classification
model = model.to(device)


In [30]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

**Training Loop**

In [23]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for data, labels in train_loader:
        data = data.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

        # Accuracy
        preds = torch.sigmoid(pred)
        predicted_labels = (preds > 0.5).float()
        train_correct += (predicted_labels == labels).sum().item()
        train_total += labels.size(0)

    avg_train_loss = train_loss / len(train_loader)
    train_accuracy = train_correct / train_total

    # Evaluation
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for data, labels in val_loader:
            data = data.to(device)
            labels = labels.float().unsqueeze(1).to(device)

            pred = model(data)
            loss = criterion(pred, labels)
            val_loss += loss.item()

            preds = torch.sigmoid(pred)
            predicted_labels = (preds > 0.5).float()
            val_correct += (predicted_labels == labels).sum().item()
            val_total += labels.size(0)

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / val_total

    print(f"Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {avg_train_loss:.4f} | Train Acc: {train_accuracy:.2%} | "
          f"Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2%}")


KeyboardInterrupt: 